# Hands-on experiment 3:  Computing a geometric embedding for the Sparsest Cut Problem via Semidefinite Programming (20 pts)

The Uniform Sparsest Cut problem (USC) aims to find a bipartition
$(S, \bar{S})$ of the nodes of a graph $G = (V, E)$,
$\vert V \vert = p$, which minimizes the quantity $$\frac{E(S, \bar{S})}{\lvert S \rvert \; \lvert \bar{S} \rvert},$$
where $E(S, \bar{S})$ is the number of edges connecting $S$ and
$\bar{S}$, and $\lvert S \rvert$ is the number of nodes in $S$. This
problem is of broad interest, with applications in areas such as VLSI
layout design, topological design of communication networks and image
segmentation. Relevant to machine learning, it appears as a subproblem
in hierarchical clustering algorithms
[@Dasgupta2016; @Chatziafratis2018].

Computing such a bipartition is NP-hard and intense research has gone
into designing efficient approximation algorithms for this problem. In
the seminal work of [@Arora2009] an $\mathcal{O}(\sqrt{\log p})$ approximation
algorithm is proposed for solving USC, which relies on finding a
*well-spread* $\ell_2^2$ geometric representation of $G$ where each node
$i\in V$ is mapped to a vector ${\textbf v}_i$ in $\mathbb{R}^p$. In this
experimental section we focus on solving the SDP that computes this
geometric embedding.

The canonical formulation of the SDP is 
\begin{align}
{\boldsymbol X}^\star \in \arg \min_{ {\boldsymbol X} } \bigg\{ \langle \boldsymbol{C}, {\boldsymbol X} \rangle : &~p \; \mathrm{Tr}({\boldsymbol X}) -  \mathrm{Tr}(\mathbf{1}_{p\times p}{\boldsymbol X}) = \frac{p^2}{2}, ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\longleftarrow ~~\equiv A({\boldsymbol X}) = \frac{p^2}{2}  \\[3mm] 
& \; {\boldsymbol X}_{i,j} + {\boldsymbol X}_{j,k} - {\boldsymbol X}_{i, k} - {\boldsymbol X}_{j,j} \leq 0, \; \forall\ i \neq j \neq k \neq i\in V, ~~~\longleftarrow ~~\equiv B_{i,j,k}({\boldsymbol X}) \in \mathcal{K} = (-\infty, 0] \tag{Problem 2} \\[3mm]
&~\underbrace{\mathrm{Tr}({\boldsymbol X}) \leq p, ~{\boldsymbol X} \in \mathbb{R}^{p\times p} ,~{\boldsymbol X}\succeq 0}_{\mathcal{X}} \bigg\},~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\longleftarrow ~~ {\boldsymbol X}  \in \mathcal{X} \text{ (the SDP cone of bounded trace)} 
\end{align}
 where $\boldsymbol{C}$ represents the Laplacian of graph
$G$ and ${\boldsymbol X}_{i,j} = \langle {\textbf v}_{i}, {\textbf v}_j\rangle$ gives the geometric
embedding of the nodes.

We can rewrite the optimization
problem <span class="reference" data-target="Problem 2">2</span> as $$
    \min_{{\boldsymbol X} \in \mathcal{X}} f({\boldsymbol X}) + g(A({\boldsymbol X})) ~~~~~~~~\text{ subject to } ~~~~B_{i,j,k}({\boldsymbol X}) \in \mathcal{K}, \;\forall\ i \neq j \neq k \neq i\in V, \tag{Problem 3}$$
where $f({\boldsymbol X}) = \langle \boldsymbol{C}, {\boldsymbol X} \rangle$ and
$g(\cdot) = \delta_{\left\{\frac{p^2}{2}\right\}}(\cdot)$ is the
indicator function of singleton $\left\{\frac{p^2}{2}\right\}$.

1.  (5 pts) How many constraints does
    <span class="reference" data-target="Problem 2">Problem 2</span> have (as a function of $p$)? How does this
    number compare to the one of
    <span class="reference" data-target="Problem 1">Problem 1</span> from Part 2?

    **N.B.1**: In Part 2 the constraints are expressed in matrix form,
    while here they are listed individually. Make sure to take this into
    account (e.g., the constraint ${\boldsymbol X} \geq 0$ in Part 2 is applied *for
    each* entry).

    **N.B.2**: You can respond to this question by either computing the
    exact number of constraints, or by identifying the correct order of
    magnitude (big-O notation).



2.  (5 pts) Express the constraints
    in <span class="reference" data-target="Problem 3">Problem 3</span> in quadratic penalty form and write down
    the corresponding penalized objective function.



3.  (10 pts) We will now observe the behavior of HCGM on three graphs
    from the Network Repository dataset [@Rossi2015]:

    -   **G1:** `mammalia-primate-association-13` with $25$ nodes,

    -   **G2:** `55n-insecta-ant-colony1-day37` with $55$ nodes and

    -   **G3:** `insecta-ant-colony4-day10` with $102$ nodes.

    You are asked to do the following:
    
    - Based on your calculation in point (1), give an estimate of the
    number of constraints for each dataset above.
    - We provide most of the code for solving this problem below. Fill in the few missing parts, run the
    algorithm for each dataset (you can cook your dinner in the
    meantime)
    - Include the generated plots in your submitted notebook. 
    - What do you notice about the running times of the algorithm for the three
    problem instances? 
    - What are the potential bottlenecks to applying
    this method to large graphs?

    One way to address the issues you identified above, especially if
    low accuracy suffices, is to resort to stochastic algorithms (the
    reasoning here is similar to the one which stands behind GD vs.
    SGD). Such an example are the methods proposed
    in [@Vladarean2020], where the framework of HCGM is used
    in conjunction with stochastic gradients and variance reduction for
    alleviating some of the shortcomings of the full-batch method you
    implemented above. A brief presentation of these methods is provided
    in the supplementary section of Lecture 13.

In [ ]:
%pip install -r Part3_requirements.txt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigsh, svds, eigs
from math import sqrt
import scipy.io as sio
import random
import numpy.matlib
import time
from scipy.sparse import isspmatrix
import itertools

from lib.part3.helpers import *

# fix the seed
random.seed( 3 )

In [ ]:
def HCGM(dataset, maxit=np.int(1e3), beta0=1e2):
    # Dataset setup
    from argparse import Namespace
    args = Namespace()
    Problem = sio.loadmat(dataset)
    args.C = np.double(Problem['LAPL']) # euclidean distance matrix
    args.p = np.int(Problem['d']) # number of data points
    args.opt_val = Problem['opt_val'] # optimum value 
    X_true = Problem['X_true']

    print("\nDataset = {}. \n".format(dataset))
    print("\n f_opt = {}. \n".format(args.opt_val))

    # Initialize
    X = np.zeros((args.p,args.p))
    
    feasibility1 = [] # norm(A1(X)-b1)/norm(b1)
    feasibility2 = [] # dist(X, \mathcal{K})
    objective    = [] # f(x)
    cur_iter    = [] 
    t    = [] 
    
    #u = np.zeros((N,1))
    iter_track = np.unique(np.ceil(np.power(2, np.linspace(0,20,50))))
    
    start = time.time()
    
    for iteration in range(1, maxit+1):
        
        # Update Step Size
        gamma = ??
        
        # Update beta
        beta_ = ??
        
        # Write down the vk to use in the lmo (eigenvalue routine)
        (vk, feas_eq, feas_ineq) = grad_F(X, beta_, args)
        # the above needs to be symmetric
        
        
        # Linear minimization oracle
        q, u = eigsh(??, k=1, tol=1e-16, which='SA')
        if q >= 0:
            X_bar = ??
        else:
            u = sqrt(args.p)*u
            X_bar = np.outer(u,u)
        
        # Update X
        X = (1.0 - gamma) * X + gamma * (X_bar)
                
        if any(iteration == iter_track) or iteration==maxit:
            feasibility1.append(feas_eq)
            feasibility2.append(feas_ineq)
            objective.append(np.sum(args.C.flatten()*X.flatten()))
            cur_iter.append(iteration)
            t.append(time.time()-start)
            print('{:03d} | {:.4e}| {:.4e}| {:.4e}|'.format(iteration, feasibility1[-1], feasibility2[-1],objective[-1]))
            
    return args, X_true, X, feasibility1, feasibility2, objective, cur_iter, t

In [ ]:

# Pick dataset
#dataset = 'data/102n-insecta-ant-colony4-day10.mat'
#dataset = 'data/55n-insecta-ant-colony1-day37.mat'
dataset = 'lib/part3/data/25mammalia-primate-association-13.mat'

args, X_true, X_HCGM, f1_HCGM, f2_HCGM, obj_HCGM, iter_HCGM, time_HCGM = HCGM(dataset, maxit=np.int(1e3), beta0=5e1)

print("\nHCGM finished. Running time: {} seconds.".format(time_HCGM[-1]))

plot_func(iter_HCGM, f1_HCGM, f2_HCGM, obj_HCGM, X_HCGM, X_true, args)

# References

<div id="refs" class="references csl-bib-body hanging-indent"
role="doc-bibliography">
<div id="ref-arv" class="csl-entry" role="doc-biblioentry">
[@Arora2009] Arora, Sanjeev, Satish Rao, and Umesh Vazirani. 2009. <span>“Expander
Flows, Geometric Embeddings and Graph Partitioning.”</span> <em>Journal
of the ACM (JACM)</em> 56 (2): 5.
</div>
<div id="ref-chatziafratis2018hierarchical" class="csl-entry"
role="doc-biblioentry">
[@Chatziafratis2018] Chatziafratis, Vaggos, Rad Niazadeh, and Moses Charikar. 2018.
<span>“Hierarchical Clustering with Structural Constraints.”</span>
<em>arXiv Preprint arXiv:1805.09476</em>.
</div>
<div id="ref-dasgupta2016cost" class="csl-entry" role="doc-biblioentry">
[@Dasgupta2016] Dasgupta, Sanjoy. 2016. <span>“A Cost Function for Similarity-Based
Hierarchical Clustering.”</span> In <em>Proceedings of the Forty-Eighth
Annual ACM Symposium on Theory of Computing</em>, 118–27.
</div>
<div id="ref-nr" class="csl-entry" role="doc-biblioentry">
[@Rossi2015] Rossi, Ryan A., and Nesreen K. Ahmed. 2015. <span>“The Network Data
Repository with Interactive Graph Analytics and Visualization.”</span>
In <em>AAAI</em>. <a
href="http://networkrepository.com">http://networkrepository.com</a>.
</div>
<div id="ref-vladarean2020conditional" class="csl-entry"
role="doc-biblioentry">
[@Vladarean2020] Vladarean, Maria-Luiza, Ahmet Alacaoglu, Ya-Ping Hsieh, and Volkan
Cevher. 2020. <span>“Conditional Gradient Methods for Stochastically
Constrained Convex Minimization.”</span> In <em>International Conference
on Machine Learning</em>, 9775–85. PMLR.
</div>
</div>